In [3]:
from pandas import *
import json
from requests import *
from numpy import *
from toucan_data_sdk.utils.generic import *

In [4]:
data_input = read_csv("../../Downloads/Agenda24072015.csv", sep=";", low_memory=False)
# populations = read_excel('../base-ic-evol-struct-pop-2015.xls', skiprows=5, usecols=['COM', 'P15_POP'])

a = get("https://raw.githubusercontent.com/rhuille/files/master/basemap/france_departement.json").json()
code_to_dep = {i["properties"]["code"]:i["properties"]["nom"] for i in a["features"]}

In [ ]:
finance = read_excel('../../Downloads/fc_2015_0.xlsx', sheet_name=""),

In [5]:
data_input.columns

Index(['idMemo', 'libelleOrg', 'ancienNom', 'typeOrganisme', 'natureOrganisme',
       'adresseComplAdresseOrg', 'adresseVoieNumeroOrg',
       'adresseVoieExtOuIndRepOrg', 'adresseVoieTypeVoieOrg',
       'adresseVoieNomVoieOrg', 'adresseMentionsDistribOrg',
       'adresseAcheminPostalCedexOrg', 'adresseAcheminPostalLibelleCedexOrg',
       'adresseCodePostalOrg', 'adresseCommuneOrg', 'adresseCodeInseeOrg',
       'regionOrg', 'paysOrg', 'latitudeOrg', 'longitudeOrg', 'libelleOffre',
       'typeOffre', 'themes', 'periodeDateDebut', 'periodeDateFin',
       'hebdoEstOuvertLundi', 'hebdoEstOuvertMardi', 'hebdoEstOuvertMercredi',
       'hebdoEstOuvertJeudi', 'hebdoEstOuvertVendredi', 'hebdoEstOuvertSamedi',
       'hebdoEstOuvertDimanche', 'horaire', 'conditions', 'tarifs', 'gratuit',
       'libelleLieu', 'adresseComplAdresseLieu', 'adresseVoieNumeroLieu',
       'adresseVoieExtOuIndRepLieu', 'adresseVoieTypeVoieLieu',
       'adresseVoieNomVoieLieu', 'adresseMentionsDistribLieu',
  

In [48]:
from pandas import *
from numpy import *

offres = {
    'SPECTACLE':'SPECTACLE',
     'FESTIVAL':'MUSIQUE',
     'MUSÉE':'MUSEE',
     'EXPOSITION':"ART",
     'VISITE':"MUSEE",
     'ATELIER':"ATELIER",
     'Conférence':"SCIENCE",
     'Circuit':"ART",
     'Portes ouvertes':"MUSEE",
     'PROJECTION':"SPECTACLE",
     'LECTURE':"LECTURE",
     'SALON':"ART"
}


MUSEE = [
    "Architecture"
]

LECTURE = ["Livre"]
 
MUSIQUE = [
#     "MUSIQUE",
"Classique et contemporaine",
"Jazz",
"Variété",
"Musiques du monde",
"Opéra",
"Baroque",
"Pop, Rock, Folk",
"Lyrique et chant choral",
"Folkloriques ou traditionnelles", 
"Classique et contemporaine",
"Électronique",
"Opérette, Opéra-bouffe",
"Sacrée",
"Rap",
"Blues"]
#          "ART",
ART = [
"Photographie",
"Arts graphiques",
"Peinture",
"Beaux-Arts",
"Arts décoratifs",
"Art contemporain",
"Mode et textiles",
"Arts de la rue",
"Multimédia",
"Design",
"Arts plastiques"]

#          "SCIENCE",
SCIENCE = [
    "Sciences humaines","Sciences de la nature","Études et recherches culturelles"]
#          "SPECTACLE",

SPECTACLE = [
    "Cinéma","Ciné-concert", 
    "Comédie musicale", "Humour", "Théâtre", "cirque", "Danse", "Jeune Public", 
    "Cabaret", "Son et lumières"]

subtheme = MUSIQUE+ART+SCIENCE+SPECTACLE+MUSEE+LECTURE

d = {}
for a in ["MUSIQUE", "ART", "SCIENCE", "SPECTACLE", 'MUSEE']:
    for i in globals()[a]:
        d[i.upper()] =a 


def extract_info(data, column, info_to_extract):
    number_of_info_extracted = Series(full(data.shape[0], 0))
    info_extracted = Series(full(data.shape[0], ""))
    column_str  = data[column].str.upper().str
    
    for t in info_to_extract:
        t = t.upper()
        info_extracted+=(number_of_info_extracted==0)*column_str.find(t).apply(lambda x: (x>-1)*t)
        number_of_info_extracted+=(column_str.find(t)>-1)
    
    info_extracted = info_extracted.replace("", NaN)
    return info_extracted
    
    
def prepare_data(data):
#     pop = (pop.groupby(['COM']).sum().reset_index().rename(columns={"COM": "CODE_POSTAL", "P15_POP": "POPULATION"}))

    data = data.assign(**{
        "REGION":lambda x: x['regionLieu'],
        "COMMUNE":lambda x: x['adresseCommuneLieu'].str.capitalize(),
        "CODE_POSTAL":lambda x: x['adresseCodeInseeLieu'],
        "DEP_CODE": lambda x: x["adresseCodePostalLieu"].str[:2],
        "SUBTHEME":lambda x: extract_info(data, "themes", subtheme),
        "THEME":lambda x: x["SUBTHEME"].replace(d),
        "TYPE":lambda x: (extract_info(data, "typeOffre", list(offres.keys()))
                          .replace(offres)),
        "DUREE": lambda x: ((to_datetime(x.periodeDateFin, format='%d/%m/%Y', errors='coerce')-
                            to_datetime(x.periodeDateDebut, format='%d/%m/%Y', errors='coerce'))
                            .dt.days.clip(1, 2*365))
    })
    
    data["THEME"] = data.THEME.fillna(data.TYPE)
    data["SUBTHEME"] = data.THEME+" > "+data.SUBTHEME.fillna("AUTRES")
    data = data[['REGION', 'COMMUNE', 'THEME', 'SUBTHEME', 'TYPE', 'DUREE', 'CODE_POSTAL', 'DEP_CODE']]
    data["DEP_NAME"] = data.DEP_CODE.replace(code_to_dep)
#     a = data.shape[0]
#     data = data.merge(pop, on='CODE_POSTAL')
#     assert a == data.shape[0], f"Le nombre de ligne n'a pas ete conserve : avant : {a} apres : {data.shape[0]} " 
    data["VALUE"]=1
    data["VALUE_GRATUIT"] = data_input.gratuit.fillna(0)
    return data

In [49]:
data = prepare_data(data_input)


In [50]:
data

,REGION,COMMUNE,THEME,SUBTHEME,TYPE,DUREE,CODE_POSTAL,DEP_CODE,DEP_NAME,VALUE,VALUE_GRATUIT
0,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,12.0,93066,93,Seine-Saint-Denis,1,0.0
1,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,8.0,93066,93,Seine-Saint-Denis,1,0.0
2,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,1.0,93066,93,Seine-Saint-Denis,1,1.0
3,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,MUSIQUE,12.0,93066,93,Seine-Saint-Denis,1,0.0
4,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,3.0,93066,93,Seine-Saint-Denis,1,0.0
5,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,8.0,93066,93,Seine-Saint-Denis,1,0.0
6,Ile-de-France,Saint-denis,SPECTACLE,SPECTACLE > CIRQUE,SPECTACLE,21.0,93066,93,Seine-Saint-Denis,1,0.0
7,Bourgogne,Louhans,MUSIQUE,MUSIQUE > RAP,MUSEE,NaN,71263,71,Saône-et-Loire,1,0.0
8,Bourgogne,Verdun-sur-le-doubs,MUSEE,MUSEE > AUTRES,MUSEE,NaN,71566,71,Saône-et-Loire,1,0.0
9,Ile-de-France,Milly-la-forêt,ART,ART > AUTRES,ART,1.0,91405,91,Essonne,1,0.0


In [41]:

(data
 .groupby("DEP_CODE").VALUE.sum().reset_index()
 .assign(LABEL = lambda x: x["DEP_CODE"].replace(code_to_dep))
).to_csv('data_map.csv', index=False)

In [ ]:
data_map = roll_up(data,
                   levels=["DEP_CODE", "SUBTHEME"],
                   groupby_vars=["VALUE", "VALUE_GRATUIT"])



data_map = data_map[["COMMUNE", "SUBTHEME", "VALUE", "type", "VALUE_GRATUIT"]]

data_map["GRATUIT_PCT"] = data_map.VALUE_GRATUIT  / data_aggregated.VALUE
data_map["DEP_CODE"] = data_map.COMMUNE.replace(dict(data[[ "COMMUNE","DEP_CODE"]].values))
data_map["DEP_NAME"] = data_map.DEP_CODE.replace(code_to_dep)


# data_aggregated.to_csv('data_aggregated.csv', index=False)

In [13]:
(DataFrame({
    "SUBTHEME":['CHOISISSEZ VOTRE ACTIVITE FAVORITE ;)']+list(data.sort_values(by="SUBTHEME").SUBTHEME.unique())})
#  .sort_values(by="SUBTHEME")
 .to_csv("subtheme_requester.csv", index=False)
)

In [14]:
(DataFrame({"COMMUNE":data.COMMUNE.unique()})
 .sort_values(by="COMMUNE")
 .to_csv("reports.csv", index=False)
)

In [51]:
data_aggregated = roll_up(data,
                          levels=["COMMUNE", "SUBTHEME"],
                          groupby_vars=["VALUE", "VALUE_GRATUIT"])



data_aggregated = data_aggregated[["COMMUNE", "SUBTHEME", "VALUE", "type", "VALUE_GRATUIT"]]

data_aggregated["GRATUIT_PCT"] = data_aggregated.VALUE_GRATUIT  / data_aggregated.VALUE
data_aggregated["DEP_CODE"] = data_aggregated.COMMUNE.replace(dict(data[[ "COMMUNE","DEP_NAME"]].values))
# data_aggregated["DEP_NAME"] = data_aggregated.DEP_CODE.replace(code_to_dep)


# data_aggregated.to_csv('data_aggregated.csv', index=False)

/anaconda3/lib/python3.6/site-packages/toucan_data_sdk/utils/generic/roll_up.py:58: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat(dfs).reset_index()


In [52]:
data_aggregated

,COMMUNE,SUBTHEME,VALUE,type,VALUE_GRATUIT,GRATUIT_PCT,DEP_CODE
0,médis,MUSIQUE > AUTRES,1,SUBTHEME,0.0,0.000000,Charente-Maritime
1,Abbeville,MUSIQUE > RAP,1,SUBTHEME,0.0,0.000000,Somme
2,Abilly,ART > AUTRES,1,SUBTHEME,0.0,0.000000,Indre-et-Loire
3,Ablon-sur-seine,ART > ARTS DE LA RUE,1,SUBTHEME,0.0,0.000000,Val-de-Marne
4,Agde,ART > ARTS DÉCORATIFS,1,SUBTHEME,0.0,0.000000,Hérault
5,Agde,MUSEE > AUTRES,1,SUBTHEME,0.0,0.000000,Hérault
6,Agde,SPECTACLE > CINÉMA,1,SUBTHEME,0.0,0.000000,Hérault
7,Agen,ART > AUTRES,3,SUBTHEME,2.0,0.666667,Lot-et-Garonne
8,Agen,ART > PEINTURE,2,SUBTHEME,2.0,1.000000,Lot-et-Garonne
9,Agen,MUSEE > AUTRES,3,SUBTHEME,0.0,0.000000,Lot-et-Garonne
